# Github Actions are Awesome!
> Or how I learned to automatically update the header image.

- toc: false
- badges: false
- comments: true
- categories: [Github, CI, Actions]
- image: images/header.png

Github Actions are a way to automate your workflow so you can build continuous-integration/continuous development (CI/CD) workflows. You can run tests, check your library or package can build, generate and commit new files all on various triggers such as pushes, pull requests or even schedules. It's really awesome for automating lots of different tasks and with scripts the possibilities are endless. It's even how this website get's built.

## Fastpages
This website is built with the amazing [fastpages](https://github.com/fastai/fastpages), which alongside [nbdev](https://nbdev.fast.ai/) is great for a gentle introduction to Github Actions.

Fastpages allows me to write this post in a [Jupyter notebook](), and then when I push these notebooks to the blog repository an action takes place that converts them to markdown files and builds the Jekyll site that you are looking at now. 

I think this is really neat but I thought it would also be cool to use this workflow to generate an ever-changing header for the front page that is based on the blog posts that have been published recently.

![title](images/header_copy.png "An example header - each post and title in the last 30 days is shown")

## An action workflow
Fastpages comes with a CI YAML file that looks something like this, which I will break down and explain and then show how I modified it so that the header is generated on both when pushing and on a schedule:

```
name: CI
on:
  push:
    branches:
      - master # need to filter here so we only deploy when there is a push to master
  # no filters on pull requests, so intentionally left blank
  pull_request:
  workflow_dispatch:

jobs:     
  build-site:
    if: ( github.event.commits[0].message != 'Initial commit' ) || github.run_number > 1
    runs-on: ubuntu-latest
    steps:
      
    - name: Check if secret exists
      if: github.event_name == 'push'
      run: |
        if [ -z "$deploy_key" ]
        then
          echo "You do not have a secret named SSH_DEPLOY_KEY.  This means you did not follow the setup instructions carefully.  Please try setting up your repo again with the right secrets."
          exit 1;
        fi
      env:
       deploy_key: ${{ secrets.SSH_DEPLOY_KEY }}
          

    - name: Copy Repository Contents
      uses: actions/checkout@main
      with:
        persist-credentials: false

    - name: convert notebooks and word docs to posts
      uses: ./_action_files

    - name: setup directories for Jekyll build
      run: |
        rm -rf _site
        sudo chmod -R 777 .
    
    - name: Jekyll build
      uses: docker://fastai/fastpages-jekyll
      with:
        args: bash -c "jekyll build -V --strict_front_matter --trace"
      env:
        JEKYLL_ENV: 'production'
        
    - name: copy CNAME file into _site if CNAME exists
      run: |
        sudo chmod -R 777 _site/
        cp CNAME _site/ 2>/dev/null || :
        
    - name: Deploy
      if: github.event_name == 'push'
      uses: peaceiris/actions-gh-pages@v3
      with:
        deploy_key: ${{ secrets.SSH_DEPLOY_KEY }}
        publish_dir: ./_site
```

The first bit sets up when the Action file gets run. The following gives the Action a name and specifies it should run on a push to master branch, or pull requests. The workflow dispatch allows manual triggering of the workflow from the actions tab or REST API on Github.

```
name: CI
on:
  push:
    branches:
      - master # need to filter here so we only deploy when there is a push to master
  # no filters on pull requests, so intentionally left blank
  pull_request:
  workflow_dispatch:
```

Next, we set up the job and what it runs on - here it is the latest ubuntu image. What this means is each time this runs we get a self-contained environment to carry out the steps in this environment. You can see below that we can sprinkle our action files with conditional `if: ...` statements allowing control flow in our actions.

```
jobs:     
  build-site:
    if: ( github.event.commits[0].message != 'Initial commit' ) || github.run_number > 1
    runs-on: ubuntu-latest
```

Now it's time to define the actual steps that we want to execute. The key part here is that we give the step a name and then either have a `run:` or `uses:` key for the given step. If we have a `run:` key we can execute statements as if we were in the bash terminal (this being a ubuntu image) such as `cd ..` or `ls` etc. 

The `use:` allows us to use (heh) specific actions created by others - of which there are many on the [GitHub Actions marketplace](https://github.com/marketplace?type=actions). You can pass arguments by using the `with:` key to these prespecified actions. 

The first step below just checks to see if we have set up a deploy key, something which should be done when we first set up a blog with fastpages:

```
    - name: Check if secret exists
      if: github.event_name == 'push'
      run: |
        if [ -z "$deploy_key" ]
        then
          echo "You do not have a secret named SSH_DEPLOY_KEY.  This means you did not follow the setup instructions carefully.  Please try setting up your repo again with the right secrets."
          exit 1;
        fi
      env:
       deploy_key: ${{ secrets.SSH_DEPLOY_KEY }}
```

Next, it uses an action to checkout our blog repository and then uses a local action directory to convert our notebooks to markdown files. This conversion puts the markdown in the `_posts/` directory of the blog repository.  

```
    - name: Copy Repository Contents
      uses: actions/checkout@main
      with:
        persist-credentials: false

    - name: convert notebooks and word docs to posts
      uses: ./_action_files
```

The final few steps build and deploy the Jekyll blog. First, the `_site/` directory is cleared and then rebuilt using a Docker container image. The CNAME step is only pertinent if you have a custom domain name, which this blog doesn't so can be ignored. 

Finally the last action `peaceiris/actions-gh-pages@v3` takes the fresh `_site/` and deploys this to a Github Page, which allows hosting of static sites. Easy right?

```
    - name: setup directories for Jekyll build
      run: |
        rm -rf _site
        sudo chmod -R 777 .
    
    - name: Jekyll build
      uses: docker://fastai/fastpages-jekyll
      with:
        args: bash -c "jekyll build -V --strict_front_matter --trace"
      env:
        JEKYLL_ENV: 'production'
        
    - name: copy CNAME file into _site if CNAME exists
      run: |
        sudo chmod -R 777 _site/
        cp CNAME _site/ 2>/dev/null || :
    
    - name: Deploy
      if: github.event_name == 'push'
      uses: peaceiris/actions-gh-pages@v3
      with:
        deploy_key: ${{ secrets.SSH_DEPLOY_KEY }}
        publish_dir: ./_site
```

## How to make the custom header image
So to make the custom header image I wrote a [little script](https://github.com/jc639/blog/blob/master/scripts/make_header.py) of which the main execution function is shown below in the collapsable code fold, but it's not too important to understand it for this. The main thing is that it reads the markdown posts in the `_posts/` folder to get the titles and dates published, and then it can construct the plot with a list of the titles and time since today's date expressed in days.

As posts only ever get put in the `_posts/` directory at site build time and the resulting markdown is not committed to the repository I realised I can slot this script into the current `ci.yaml` workflow.

In [2]:
#collapse
def line_plot(post_titles: list, deltas: list, cutoff=-30):
    """Creates the line plot in the XKCD style.

    Args:
        post_titles (list): list of post titles
        deltas (list): list of time since comparison time
        cutoff (int, optional): cutoff point. Defaults to -30.

    Returns:
        tuple : plt.fig, plt.ax
    """
    with plt.xkcd():
        f, ax = plt.subplots(1, 1)
        x_vals = [i for i in range(cutoff, 0, 1)]
        y_counts = Counter(deltas)
        y_vals = [y_counts[x] for x in x_vals]
        ax.plot(x_vals, y_vals)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_title('WELCOME TO THE BLOG!', fontweight='bold', y=1.05)
        max_y_val = max(y_vals)
        ax.set_ylim(top=max_y_val+0.3+0.2*len(y_counts))
        ax.set_yticks(range(0, max_y_val+1))
        ax.set_xlabel('Days ago...')
        ax.set_ylabel('Number of posts')

        arrowprops = dict(
                        arrowstyle="->",
                        connectionstyle="angle3,angleA=0,angleB=90")
        titles_arr = np.array(post_titles)
        deltas_arr = np.array(deltas)
        x_offset = 2
        y_offset = max_y_val + 0.2*len(y_counts) + 0.2
        for x, count in y_counts.items():
            post_titles = titles_arr[deltas_arr == x]
            ax.annotate('\n+\n'.join(post_titles), xy=(x, count),
                        xytext=(x+x_offset, y_offset),
                        arrowprops=arrowprops)
            y_offset -= 0.2

        ax.text(x=1.2, y=0.6, s='Days since posting...',
                transform=ax.transAxes)
        bbox_props = dict(boxstyle="round", fc="white", ec="black")
        if len(deltas) > 0:
            last_post_days = str(abs(deltas[0]) - 1)
        else:
            last_post_days = '+' + str(abs(cutoff))
        ax.text(1.3, 0.4, last_post_days, bbox=bbox_props, transform=ax.transAxes,
                fontsize=24)
        exclam = '"Nice!"' if int(last_post_days) < 14 else '"UH OH!"'
        ax.text(1.28, 0.1, exclam, transform=ax.transAxes, rotation=25)
        f.set_size_inches(12, 2.5)
        return f, ax

Heres the changes I made to make that work.

Firstly I had to add another trigger to the action. It's fine that it runs on push as I want new blog posts to be added to the header as they are published but I also want it to update every day so that the time since publishing updates daily for each post. Handily GitHub actions have a `schedule` trigger, where we set a schedule for the action to occur using Cron expressions. 

```
name: CI
on:
  push:
    branches:
      - master # need to filter here so we only deploy when there is a push to master
  # no filters on pull requests, so intentionally left blank
  pull_request:
  workflow_dispatch:
  schedule:
    - cron: "0 1 * * *"
```

This Cron expression means the action will happen every day at 1 am. Check this handy website if you need to write Cron expression https://crontab.guru/.

Next, I slot the script and associated setup in between `- name: convert notebooks and word docs to posts` and the `- name: setup directories for Jekyll build steps`. To use the script we need to have python and the required libraries installed. Luckily this is again quite easy using the `actions/setup-python` action followed by a `run: pip install -r requirements.txt` step.

The next step installs the humor sans font that is required by `plt.xkcd` and then finally we run the script.

```
    - name: convert notebooks and word docs to posts
      uses: ./_action_files

    - name: setup python
      uses: actions/setup-python@v2
      with:
        python-version: 3.7

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Install humor sans font
      run: |
        sudo apt-get update -y
        sudo apt-get install -y fonts-humor-sans
        rm -rf ~/.cache/matplotlib

    - name: make-header
      run: |
        python scripts/make_header.py

    - name: setup directories for Jekyll build
      run: |
        rm -rf _site
        sudo chmod -R 777 .
```

The very last thing we have to change is the conditional `if:` in the final `- name: Deploy` step to also run this when the `github.event_name` is equal to 'schedule'. Viola!

```
    - name: Deploy
      if: ( github.event_name == 'push' ) || ( github.event_name == 'schedule' )
      uses: peaceiris/actions-gh-pages@v3
      with:
        deploy_key: ${{ secrets.SSH_DEPLOY_KEY }}
        publish_dir: ./_site
```